<a href="https://colab.research.google.com/github/mdaugherity/MachineLearning2024/blob/main/class/HW3_Template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HW3 - Trees on the Titanic

Name: **YOUR NAME HERE**  
Class: PHYS453 - Machine Learning  
Date: Spring 2024  






In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.datasets import fetch_openml
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

# Input
Load and process the input data.  

*Use the code below to load data without modifications.*

In [ ]:
# Load Titanic
data = fetch_openml(name="titanic",version=1, as_frame=True, parser='auto')

df_raw = data.frame # the raw data
df_raw.head()

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0,0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"
2,1,0,"Allison, Miss. Helen Loraine",female,2.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1,2,113781,151.5500,C22 C26,S,NaN,135.0,"Montreal, PQ / Chesterville, ON"
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"


In [ ]:
# Factorize any non-numeric columns we want to use
codes, genders = pd.factorize(df_raw.sex)
df_raw['gender'] = codes
print('Gender codes:', genders.categories)

Gender codes: Index(['female', 'male'], dtype='object')


In [ ]:
print('Original data:')
print(df_raw.sex.value_counts())

print('\nFactorized result:')
df_raw.gender.value_counts()

Original data:
male      843
female    466
Name: sex, dtype: int64

Factorized result:


1    843
0    466
Name: gender, dtype: int64

In [ ]:
# Choose columns to keep
df_raw.columns

Index(['pclass', 'survived', 'name', 'sex', 'age', 'sibsp', 'parch', 'ticket',
       'fare', 'cabin', 'embarked', 'boat', 'body', 'home.dest', 'gender'],
      dtype='object')

In [ ]:
features = ['pclass', 'gender', 'age', 'sibsp', 'parch', 'fare'] # choose columns for features
target = ['survived']
cols = target + features # combination of target and features
df = df_raw[cols].copy()
df.head()

,survived,pclass,gender,age,sibsp,parch,fare
0,1,1,0,29.0000,0,0,211.3375
1,1,1,1,0.9167,1,2,151.5500
2,0,1,0,2.0000,1,2,151.5500
3,0,1,1,30.0000,1,2,151.5500
4,0,1,0,25.0000,1,2,151.5500


In [ ]:
df.describe()

,pclass,gender,age,sibsp,parch,fare
count,1309.000000,1309.000000,1046.000000,1309.000000,1309.000000,1308.000000
mean,2.294882,0.644003,29.881135,0.498854,0.385027,33.295479
std,0.837836,0.478997,14.413500,1.041658,0.865560,51.758668
min,1.000000,0.000000,0.166700,0.000000,0.000000,0.000000
25%,2.000000,0.000000,21.000000,0.000000,0.000000,7.895800
50%,3.000000,1.000000,28.000000,0.000000,0.000000,14.454200
75%,3.000000,1.000000,39.000000,1.000000,0.000000,31.275000
max,3.000000,1.000000,80.000000,8.000000,9.000000,512.329200


WARNING!  The count is different for different columns.  We must have some missing data!

In [ ]:
# Clean up dataframe by dropping missing rows
print('Row count:\t', len(df))
df.dropna(inplace=True)  # delete rows with missing or bad values
print('After dropna:\t', len(df))

Row count:	 1309
After dropna:	 1045


In [ ]:
# Save variables
X = df[features].values
y = df[target].values

# Load passenger data

In [ ]:
df_pass = pd.read_csv('https://raw.githubusercontent.com/mdaugherity/MachineLearning2024/main/data/titanic_passengers.csv')
df_pass

,Name,pclass,gender,age,sibsp,parch,fare
0,Dawson,3,1,49,3,3,58.5
1,Ziyu,2,1,83,4,1,60.5
2,Colton,3,1,21,0,0,1.0
3,Kathryn,2,0,20,1,3,200.0
4,Mirimo,1,1,36,3,0,300.0
5,Sidney,3,1,26,0,0,0.0
6,Justin,1,1,76,4,1,77.0
7,Mike,1,1,99,1,17,250.0


In [ ]:
X_pred = df_pass[features] # make sure we get the same feature columns in the same order
X_pred

,pclass,gender,age,sibsp,parch,fare
0,3,1,49,3,3,58.5
1,2,1,83,4,1,60.5
2,3,1,21,0,0,1.0
3,2,0,20,1,3,200.0
4,1,1,36,3,0,300.0
5,3,1,26,0,0,0.0
6,1,1,76,4,1,77.0
7,1,1,99,1,17,250.0


# Testing



## Problem 1

* Choose any 2 features and make a new variable called ```X2``` with only these 2 columns
* Train a tree using all rows of ```X2``` and ```y``` with max_depth=3
* Print a nice diagram of the tree.  Make sure it is filled and all features and classes are labeled!
* Make a plot of the decision boundaries and verify it matches the diagram

# Training

## Problem 2
Using all 6 features, find the optimal value of max_depth.

# Predicting


## Problem 3

Use the **best** tree to classify the extra passengers.  

# Discussion

**DISCUSSION GOES HERE**